In [1]:
import codebase.web_scrape_functions as wsf
import codebase.analysis_functions as af
import codebase.match_data as match_data
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import utils
from utils import logger
import os
import logging
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import codebase.graphing_functions as gf
from codebase.settings import CAREERS
from codebase.settings import LABEL_DATA

if utils.check_if_ipython():
    logger.disabled = True

%load_ext autoreload
%autoreload 2
logger.handlers[1].setLevel(logging.INFO)
# logger.disabled = True
pd.get_option("display.max_columns")

20

First step is to get Kohli's career averges for all shots, then his averages for just specific shots and then we can find his most productive shot. The problem with searching things based off of specific shots strings only is that there is a veriety in the way that shots are described in the commentary. We would need a machine learning algorithm to work and recognize what these shots are. But first we need to create a training set which is a pain in the ass.

In [2]:
KOHLI_ID = '253802'
ROOT_PLAYER_ID = '303669'
WILLIAMSON_PLAYER_ID = '277906'
SPD_SMITH_ID = '267192'

In [3]:
kohli_matches = wsf.get_player_match_list(KOHLI_ID)

In [4]:
kohli_innings = af.get_cricket_totals(KOHLI_ID, kohli_matches, _type='bat', by_innings=True, is_object_id=True, from_scorecards=False)

In [5]:
kohli_comms = af.get_player_contributions(KOHLI_ID, kohli_matches, _type='bat', by_innings=True, is_object_id=True)

In [6]:
kohli_comms_flat = [x for comms in kohli_comms for x in comms.commentTextItems.to_list()]

In [7]:
def search_shots_in_comms(contributions, search_keywords, exlude_words=[], primary_keywords=[], threshold = 0.5):
    shots = []
    for i, comms in enumerate(contributions):
        innings = comms.commentTextItems.to_list()
        search = af.search_for_phrases(innings, keywords=search_keywords, exclude_words=exlude_words, primary_keywords=primary_keywords, threshold = threshold)
        shots.append(comms.iloc[[i for i,x in enumerate(search[0]) if x == 1]])
        
    return shots

In [ ]:
## Cover drives 
cover_drive_primary_kw = ['cover drive', 'cover-drive']

cover_drive_search_kw = [
    'drive',
    ('punch', 0.3),
    ('cover', 0.3),
    ('full', 0.2),
    ('wide', 0.2) ,
    'outside edge',
    ('reach', 0.3),
    'slip',
    'edge',
    'driving'
]
cover_drive_exlude_words = [
    'run out',
    'pull', 'flick',
    'bouncer', 'short ball', 'stays back',
    'backfoot', 'top edge', 'leave',
    'lets one go', 'easy leave', 'leaves the ball',
    'leg side', 'leading edge', 'leg ', 'cut', 'leg.',
    'on the pads', 'left alone', ('short', 0.25)
]


In [8]:
cover_drive_primary_kw = ['cover drive', 'cover-drive']

cover_drive_search_kw = [
    'drive',
    ('punch', 0.3),
    ('cover', 0.3),
    ('off side', 0.2),
    ('full', 0.25),
    ('wide', 0.2) ,
    ('outside edge', 0.3),
    ('reach', 0.3),
    ('slip', 0.25),
    ('edge', 0.3),
    'driving',
    ('force', 0.2),
    ('push', 0.3),
    ('pitched up', 0.3),
    ('Overpitched', 0.2),
    ('dab', 0.2),
    ('outside off', 0.25)

]
cover_drive_exlude_words = [
    'run out',
    'pull', 'flick',
    'bouncer', 'short ball', 'stays back',
    'backfoot', 'top edge', 'leave', 'back foot', ('back', 0.2),
    'lets one go', 'easy leave', 'leaves the ball', 'goes back',
    'leg side', 'leading edge', ('leg ', 0.25), 'cut', ('leg', 0.25),
    'on the pads', 'left alone', ('short', 0.25), ('midwicket', 0.2), ('long-on', 0.25), ('long on', 0.25), ('back of a length', 0.25)
]



In [9]:
ind, scores = af.search_for_phrases(kohli_comms_flat, cover_drive_search_kw, cover_drive_exlude_words, primary_keywords=cover_drive_primary_kw, threshold=0.49)
cover_drives_career = [kohli_comms_flat[i] for i,x in enumerate(ind) if x==1]
cover_drives_almost = [kohli_comms_flat[i] for i,x in enumerate(scores) if x < 0.49 and x > 0]

In [ ]:
print(len(cover_drives_career))
cover_drives_career

In [ ]:
print(len(cover_drives_almost))
cover_drives_almost[1000:]

In [11]:
cover_drives_played = search_shots_in_comms(kohli_comms, cover_drive_search_kw, cover_drive_exlude_words, primary_keywords=cover_drive_primary_kw)

In [12]:
cover_drive_averages, cover_drive_totals, cover_drive_stats = af.analyse_batting(cover_drives_played)

In [ ]:
cover_drive_averages

In [ ]:
cover_drive_totals

In [ ]:
flick_primary_kw = [
    'flick'
]
flick_search_kw = [
    'tuck',
    'clipped',
    'clip',
    ('pads', 0.3),
    ('pad ', 0.3),
    'on the pads',
    #('leg side', 0.25),
    ('leg', 0.3),
    ('square', 0.19),
    'glance'
    'off the pads',
    'off his pads',
    ('nudge', 0.3),
    ('pick', 0.3),
    ('straight', 0.3),
    #('square leg', 0.25),
    ('whip', 0.4),
    'on the legs',
    ('work', 0.25),
    ('midwicket', 0.3),
    ('on side', 0.25),
    ('across', 0.3)
]
flick_exlude_words = [
    'run out', 'bumper', 'sweep', 'swept',
    'pull', 'drive', 'cover-drive', ('cover', 0.3),
    'bouncer', 'short ball', 'stays back',
    'backfoot', 'top edge',
    'lets one go', 'easy leave', 'leaves the ball', 'padded away',
    ('off side', 0.25), 'cut', 'left alone', ('point', 0.25), ('defend', 0.3), ('push', 0.25)
]

flick_shots_career = search_shots_in_comms(kohli_comms, flick_search_kw, flick_exlude_words, flick_primary_kw, threshold=0.49)

In [ ]:
flick_averages, flick_totals, flick_stats = af.analyse_batting(flick_shots_career)

In [ ]:
flick_averages
#Flicks mostly produce singles from Kohli, there are almost as many ones as there are dots with flicks, so this is a genrally 'safe' shot for Kohli which is quite productive.

In [ ]:
flick_totals

In [ ]:
pull_primary_kw = ['short-arm jab']

pull_search_kw = [
    'pull',
    'hook',
    ('short', 0.2),
    ('swivel', 0.3),
    ('bouncer', 0.2),
    ('bumper', 0.2),
    ('hammer', 0.3),
    ('slam',  0.3),
    ('slap',  0.3),
    ('thump', 0.3),
    ('smash', 0.3),
    ('pummels', 0.3)

]
pull_exlude_words = [
    'full',
    'cut',
    'flick',
    'drive',
    'push',
    'block',
    'outside edge',
    ('cover', 0.3),
    ('pads', 0.3), ('off side', 0.25), ('point', 0.25),
    'lets one go', 'easy leave', 'leaves the ball', 'left alone', 'lets it go', 'inside edge'
    
]

In [ ]:
pull_ind, pull_scores = af.search_for_phrases(kohli_comms_flat, pull_search_kw, pull_exlude_words, primary_keywords=pull_primary_kw, threshold=0.49)
pulls_career = [(kohli_comms_flat[i], i) for i,x in enumerate(pull_ind) if x==1]
pulls_almost = [(kohli_comms_flat[i], i) for i,x in enumerate(pull_scores) if x < 0.49 and x > 0]

In [ ]:
pulls_career

In [ ]:
pulls_almost

In [ ]:
pull_shots_career = search_shots_in_comms(kohli_comms, pull_search_kw, pull_exlude_words, primary_keywords=pull_primary_kw, threshold=0.49)
pull_averages, pull_totals, pull_stats = af.analyse_batting(pull_shots_career)

In [ ]:
pull_averages

In [ ]:
pull_totals

In [8]:
cut_primary_kw = [
]
cut_search_kw = [
    ('cut', 0.6),
    ('point', 0.4),
    ('guide', 0.3),
    ('cover', 0.1),
    ('third man', 0.2),
    ('short', 0.2),
    ('wide', 0.2),
    ('bouncer', 0.2),
    ('bumper', 0.3)
    
]
cut_exlude_words = [
    'full', 'pull', 'hook', 'flick', 'drive', ('leg', 0.3), ('pads', 0.3), ('cut'), 'defend', 'defensive', 'leaving alone',
    'lets one go', 'easy leave', 'leaves the ball', 'left alone', 'pitching', 'lets it go', 'inside edge', ('cutter', 0.3), 'pitched up'
]

In [9]:
cut_ind, cut_scores = af.search_for_phrases(kohli_comms_flat, cut_search_kw, cut_exlude_words, primary_keywords=cut_primary_kw, threshold=0.55)
cuts_career = [(kohli_comms_flat[i],i) for i,x in enumerate(cut_ind) if x==1]
cuts_almost = [(kohli_comms_flat[i],i) for i,x in enumerate(cut_scores) if x < 0.55 and x > 0]

In [10]:
cuts_career

[('Turns away from a shortish length and virat goes back to push it towards point',
  1),
 ('Short of a length outside off, kohli stays back and looks to guide through backward point.,',
  94),
 ('Short of a length outside off, kohli guides easily through covers for a single.',
  121),
 ('This one breaks away from kohli as he leans out, and he adjusts to guide the push into the point area.',
  128),
 ('Bishoo keeps it short of a length outside off, and kohli works it towards cover point.',
  153),
 ('Goes on the back foot to guide it towards backward point', 202),
 ('Short of a length, outside off, pushed to deep point to get to his personal best. the distance to the follow-on mark under 20 now',
  294),
 ('Shorter ball this time, just outside off, kohli controls that one well, jumps to get on top of the bounce and taps the ball towards point',
  309),
 ('Shorter and well outside off, slashed towards deep point for a single',
  322),
 ('Length ball just outside off, guided towards back

In [11]:
cuts_almost

[('There arrives the short ball, but a touch outside off stump and virat calmly weaves out of its path',
  6),
 ('Gets behind a short of a length ball and blocks it back to the bowler', 7),
 ('Punched away to cover', 17),
 ('Stays back of a length, kohli wants a quick single but he has hit it straight to cover',
  28),
 ('In-between length, pushed away to cover', 33),
 ('Bouncer, too short really, easy for kohli to duck under', 34),
 ('Wide of the crease, a length ball angled in, kohli is solid in defence',
  36),
 ('Good bouncer, in the chest-throat area, fended past short leg', 41),
 ('This bouncer bounces as high as expected, and he can duck under it', 43),
 ('Short, kohli fends with the bottom hand off, straight into the ground. more staring',
  45),
 ('Quick quick bouncer, 92.2mph, off-stump line, and kohli drops his wrists and sways out of its line',
  50),
 ('Right behind this one, drops it towards cover', 58),
 ('Nicely guided past the hobbling bishoo at mid-on', 59),
 ('Wide o

In [45]:
cut_shots_career = search_shots_in_comms(kohli_comms, cut_search_kw, cut_exlude_words, primary_keywords=cut_primary_kw, threshold=0.55)
cut_averages, cut_totals, cut_stats = af.analyse_batting(cut_shots_career)

In [46]:
cut_averages

{'runs': 1.52,
 'dismissals': 0.0,
 'balls': 1.41,
 'sr': 0.55,
 'average': nan,
 'dot_balls': 0.57,
 'ones': 0.52,
 'twos': 0.13,
 'threes': 0.03,
 'fours': 0.16,
 'fives': 0.0,
 'sixes': 0.0,
 'how-out': 0.0,
 'total_balls_faced': 50.02,
 'fours_per_ball': 0.07,
 'sixes_per_ball': 0.0,
 'dots_per_ball': 0.22}

In [47]:
cut_totals

{'runs': 263,
 'dismissals': 0,
 'balls': 244,
 'sr': 94.95,
 'average': nan,
 'dot_balls': 98,
 'ones': 90,
 'twos': 23,
 'threes': 5,
 'fours': 28,
 'fives': 0,
 'sixes': 0,
 'how-out': 0,
 'total_balls_faced': 8654.0,
 'fours_per_ball': 11.64,
 'sixes_per_ball': 0.0,
 'dots_per_ball': 38.830000000000005}

In [23]:
sweep_primary_kw = [
]
sweep_search_kw = [
    'paddle',
    'sweep',
    
]
sweep_exlude_words = [
    'sweeper', 'drive', 'cut', 'flick', 'pull'
]

In [24]:
sweep_ind, sweep_scores = af.search_for_phrases(kohli_comms_flat, sweep_search_kw, sweep_exlude_words, primary_keywords=sweep_primary_kw, threshold=0.49)
sweep_career = [(kohli_comms_flat[i],i) for i,x in enumerate(sweep_ind) if x==1]
sweep_almost = [(kohli_comms_flat[i],i) for i,x in enumerate(sweep_scores) if x < 0.49 and x > 0]

In [25]:
sweep_career

[('Attacking sweep shot from a straight line to deep backward square. bounces once before it reaches the fielder',
  3644),
 ('Length and outside off, kohli watches the ball carefully, weathers a bit of extra bounce and he gets a top edge that goes to the  square leg boundary. unconvincing sweep',
  3914),
 ("Top edge to the sweep, but it doesn't carry to short fine leg. lyon's bounce is troubling everyone, as our experts said",
  3922),
 ("Nails the sweep this time and he's found the square leg boundary. seemed a touch too full and just enough room, kohli's strong bottom hand took care of the challenge very well.",
  3932),
 ("Carted it over the leg side infield! it's pretty well populated there with midwickets and square legs and fine legs, but he sends it soaring over all of them with a powerful slog sweep. and even that has a whip to it!",
  3984),
 ('Flighted outside off, kohli looks to sweep and gets struck on the helmet. bounces off the grille to short fine leg',
  3994),
 ("Fli

In [26]:
sweep_almost

[('Crouches and bends the knee to work a flatter ball in front of square leg',
  204),
 ('Bends the knee and avoids the bouncer without any fuss', 1196),
 ('Shortish and outside off, kohli gets on his knees as he makes sure he leaves the ball, steyn has more advice for kohli',
  2646),
 ('143.4 kph, shortish and rising as kohli bends his knees and lets it go',
  2789),
 ('Gets down low, almost on one knee as he came forward to block', 3071),
 ('This time kohli sinks to his knees under the short ball. got the height he wanted, southee as this sails through to the keeper only a shade outside off stump. no room either',
  3235),
 ('144 kph, sinks to one knee under the bouncer. good height from boult and it was very quick too.',
  3246),
 ('Short ball just a shade outside off, no threat to kohli who has has sunk to one knee',
  3314),
 ('Short ball over off stump, kohli bends his knees and drops his hands',
  3473),
 ('Terrific ball! banged in short and it follows the batsman. kohli tries 

In [41]:
sweep_shots_career = search_shots_in_comms(kohli_comms, sweep_search_kw, sweep_exlude_words, primary_keywords=sweep_primary_kw, threshold=0.49)
sweep_averages, sweep_totals, sweep_stats = af.analyse_batting(sweep_shots_career)

In [42]:
len(sweep_shots_career)

173

In [43]:
sweep_averages

{'runs': 0.45,
 'dismissals': 0.02,
 'balls': 0.24,
 'sr': 0.22,
 'average': nan,
 'dot_balls': 0.1,
 'ones': 0.03,
 'twos': 0.02,
 'threes': 0.01,
 'fours': 0.08,
 'fives': 0.0,
 'sixes': 0.01,
 'total_balls_faced': 15.97,
 'fours_per_ball': 0.04,
 'sixes_per_ball': 0.01,
 'dots_per_ball': 0.05}

In [44]:
sweep_totals

{'runs': 77,
 'dismissals': 4,
 'balls': 42,
 'sr': 37.42,
 'average': nan,
 'dot_balls': 17,
 'ones': 6,
 'twos': 3,
 'threes': 1,
 'fours': 14,
 'fives': 0,
 'sixes': 1,
 'total_balls_faced': 2763.0,
 'fours_per_ball': 6.53,
 'sixes_per_ball': 1.0,
 'dots_per_ball': 9.39}

In [8]:
backfoot_drive_primary_kw = [
]
backfoot_drive_kw = [
    ('drive', 0.3),
    ('back foot'),
    ('punch', 0.3),
    #('cover', 0.3),
    #('off side', 0.2),
    ('goes back', 0.3), ('stays back', 0.3), ('gets back', 0.3), ('rocks back', 0.3), ('go back', 0.3),
    ('short', 0.3),
    ('of a length', 0.2),
    #('outside edge', 0.3),
    #('reach', 0.3),
    #('slip', 0.25),
    #('edge', 0.3),
    ('driving', 0.3),
    ('force', 0.2),
    ('push', 0.3),
    ('waft', 0.2),
    #('dab', 0.2),
    #('outside off', 0.25)
    
]
backfoot_drive_exclude_words = [
    'run out', 'over-pitched', 'overpitched',
    'pull', 'flick', 'whip', ('duck', 0.3),
    'forward', ('length ball', 0.2), ('guide', 0.3),
    'full toss', ('full', 0.8), 'comes forward', ('defend'), 'defense',
    'backfoot', 'top edge', 'leave', 'front foot', ('front', 0.2),
    'lets one go', 'easy leave', 'leaves the ball', 'goes forward',
    'leg side', 'leading edge', ('leg ', 0.25), 'cut', ('leg', 0.25), 'weave'
    'on the pads', 'left alone', ('midwicket', 0.3), ('long-on', 0.25), ('long on', 0.25)
]


In [9]:
backfoot_drive_ind, backfoot_drive_scores = af.search_for_phrases(kohli_comms_flat, backfoot_drive_kw, backfoot_drive_exclude_words, primary_keywords=backfoot_drive_primary_kw, threshold=0.55)
backfoot_drive_career = [(kohli_comms_flat[i],i, backfoot_drive_scores[i]) for i,x in enumerate(backfoot_drive_ind) if x==1]
backfoot_drive_almost = [(kohli_comms_flat[i],i, x) for i,x in enumerate(backfoot_drive_scores) if x < 0.49 and x > 0]

In [15]:
backfoot_drive_career

[('Turns away from a shortish length and virat goes back to push it towards point',
  1,
  0.8999999999999999),
 ('Gets behind a short of a length ball and blocks it back to the bowler',
  7,
  0.6000000000000001),
 ('Back of a length, sharp, outside off, pushed to off', 27, 0.8),
 ('Pitches leg, turns away, but it slightly short, and kohli deals easily off the back foot',
  31,
  0.75),
 ('Another short one, now that he has thrown a punch, kohli decides to go back to ducking under it.',
  85,
  0.8999999999999999),
 ('Short of a length outside off, kohli stays back and looks to guide through backward point.,',
  94,
  0.8),
 ("Short of a length from sammy, who is looking to hold up an end till the new ball is taken, i reckon. kohli doesn't try to force the issue.",
  130,
  0.7),
 ("Kohli's day just got worse, a loose drive away from his body, it was short of a length and outside off, perhaps should have left it alone, he decides to drive on the up instead, though edwards has been swi

In [10]:
backfoot_drive_almost

[('There arrives the short ball, but a touch outside off stump and virat calmly weaves out of its path',
  6,
  0.3),
 ('Driven to bravo at cover', 14, 0.3),
 ('Punched away to cover', 17, 0.3),
 ('Brute! absolute brute. hits a crack on a length, and jumps at his, beating his drive. dravid comes up and has a word with him',
  25,
  0.3),
 ('Nice inside-out drive wide of mid-off for one', 32, 0.3),
 ('In-between length, pushed away to cover', 33, 0.3),
 ('Good bouncer, in the chest-throat area, fended past short leg',
  41,
  0.04999999999999999),
 ('Short, kohli fends with the bottom hand off, straight into the ground. more staring',
  45,
  0.3),
 ("Another ball that keeps low, but this also jagged back in, and kohli was lucky it didn't come back in enough. bounces in front of baugh and beats him",
  52,
  0.09999999999999998),
 ('Angling in towards the stumps, pushed square on the on side', 56, 0.3),
 ('Short, outside off, defended solidly. edwards over-stepped too', 60, 0.3),
 ('Ano